<a href="https://colab.research.google.com/github/youngmoo/ECES-434/blob/main/Class%204.2%20(2021-02-03).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-434: Class 4.2 (2021-02-03)**
Week 4: Groundhog says 6 more weeks of Winter :-(

As always, start by importing the "usual" modules we'll be using...

In [ ]:
import numpy as np                      # NumPy, abbreviated as np
import matplotlib.pyplot as plt         # MatplotLib PyPlot module, abbreviated as plt
from matplotlib import animation, rc    # MatplotLib animation module
%matplotlib inline
from scipy import signal                # SciPy's signal module, for DSP functions
import soundfile as sf                  # Switching to the soundfile module for reading and writing soundfiles

import IPython.display as ipd           # Interactive Python display module, for playing sounds
from IPython.display import HTML        # For displaying animations
rc('animation', html='jshtml')          # Provides animation controls

ClassPath = '/content/drive/My Drive/ECES-434 Sessions/Class 4-2/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'anonymous'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Custom plotting functions
Because we plot a lot... Minor tweaks since last class (4.1)

## plotSpectrogram

In [ ]:
def plotSpectrogram(sig, fs, win='hann', nseg=512, olap=256, fft_len=512):
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=nseg, noverlap=olap, nfft=fft_len)

  fig = plt.figure(figsize=(16,6))

  plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlot(): properly formats time domain plot of a signal

In [ ]:
def myPlot(sig, fs=44100):
  fig = plt.figure(figsize=(16,4))
  t = np.arange(len(sig)) / fs
  plt.plot(t, sig)
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlotFFT(): properly formats frequency domain plot of a signal

In [ ]:
def myPlotFFT(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, 20*np.log10(np.abs(S)))
  plt.xlim(0, x_lim)
  return fig, plt  

## Custom FFT animation functions

In [ ]:
n_o = 0
f_size = 2048
n_hop = f_size / 2
N_fft = 4096
fs = 44100
f = np.arange(N_fft) * fs / N_fft

# First set up the figure, the axis, and the plot element we want to animate
def setupAnimFFT(x_lim=(0,20000), y_lim=(-120,100)):
  fig = plt.figure(figsize=(14,6))
  ax = plt.axes(xlim=x_lim,ylim=y_lim)
  plt.close()   # Don't output the final figure separately
  line, = ax.plot([], [])
  return fig, line

# initialization function: plot the background of each frame
def initAnimFFT():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially  
def animateFFT(i, sig):
    n1 = int(n_o + n_hop*i)
    n2 = int(n_o + n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_mag = 20*np.log(np.abs(X_i))

    line.set_data(f, X_mag)
    return (line,)  

# Usage:
# fig, line = setupAnimFFT()
# anim = animation.FuncAnimation(fig, animateFFT, init_func=initAnimFFT, frames=120, fargs=(signal,), interval=1000/30, blit=True)
# anim

# Load today's sound files

In [ ]:
a44s, fs44 = sf.read(ClassPath + 'sounds/Haydn-44kHz.wav')
a44 = np.mean(a44s, axis=1)
ipd.Audio(a44,rate=fs44)

In [ ]:
a11, fs11 = sf.read(ClassPath + 'sounds/Haydn-11kHz.wav')
ipd.Audio(a11,rate=fs11)

# Continuing (almost) arbitrary sample rate conversation
We were changing the sampling rate of a signal by a rational number (a ratio of integers)

## Last class: Upsample and downsample by intgers, even large factors

In [ ]:
a11_up640 = np.zeros(len(a11)*640)
a11_up640[::640] = a11
myPlot(a11_up640[:10000])

## Let's scale back our ambitions, and upsample/downsample by smaller factors?
We ran into computational issues with large factors, so let's start with 7/5
* 11025 (7/5) = 15435

First upsample by 7...

In [ ]:
a11_up7 = np.zeros(len(a11)*7)
a11_up7[::7] = a11
myPlot(a11_up7[:500])

Take a look at the spectrum of the first part of the upsampled signal, say 16384 samples (remember, we're now at a sampling rate of:
* 11025(7) = 77175

In [ ]:
myPlotFFT                     # What are we looking at?

### And filter with a straight moving average...

In [ ]:
L = 13                     # Length (order) of our moving average FIR filter
h_ma = np.ones(L)/L
a11_up7_ma =               # How do we convolve our signal with this FIR filter?

myPlot(a11_up7_ma[:500])   # Plot the signal to take a look

### Take a look at the straight MA filter in the frequency domain...

In [ ]:
myPlotFFT

### Take a look at the beginning of our filtered signal in the frequency domain...

In [ ]:
myPlotFFT

### Downsample the MA filtered signal and take a listen...

In [ ]:
ipd.Audio(a11_up7_ma[::5],rate = 15435)

## What if we try a weighted, moving average, like a Hann window?

In [ ]:
M = 13                              # Length of our Hann-weighted FIR filter
h_hann = np.hanning(M)              # Use the Hann(ing) function
h_hann = h_hann / np.sum(h_hann)    # Normalize the filter
myPlot(h_hann, fs=11025*7)          # Plot our filter

In [ ]:
a11_up7_hann =                         # Convolve our signal with the filter
myPlot(a11_up7_hann[:1000])

### Plot the FFT of the Hann filter

In [ ]:
myPlotFFT

### Plot the FFT of the filtered signal
Not the whole signal, just the beginning, maybe 16384 samples.

In [ ]:
myPlotFFT

### Downsample by 5 and take a listen

In [ ]:
a15435 = a11_up7_hann[::5]
myPlot(a15435[:1000])
ipd.Audio(a15435,rate=15435)

## Try other windows?
* Bartlet (triangle): Linear interpolation
* Hamming
* Blackman

In [ ]:
M =                               # Set the filter order (length)
h_mine =                          # define a window
h_mine =                          # Normalize the window

a11_up7_mine =                    # Convolve upsampled signal with your filter

fig,plt = myPlotFFT               # Plot the FFT of your new filter

# Uncomment next line to save your figure to Google Drive
# fig.savefig(ClassPath + 'filters/' + username + '-filter.png')

### Plot the filtered signal

In [ ]:
a11_up7_mine =   # Convolve upsampled signal with your filter

fig,plt =        # Plot the filtered signal

# Uncomment next line to save figure to Google Drive
# fig.savefig(ClassPath + 'signals/' + username + '-signal.png')

### Plot the FFT of the filtered signal

In [ ]:
fig,plt = myPlotFFT

# fig.savefig(ClassPath + 'signal_FFTs/' + username + '-signal_FFT.png')

# What about an IIR filter?


In [ ]:
A = 0.5
h_iir = (1-A) * A ** np.arange(2048)
myPlot(h_iir)

In [ ]:
myPlotFFT(h_iir, x_lim=40000, fs=fs11*7)

In [ ]:
def myPlotFFTPhase(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  
  plt.plot(f,                          # How do we get the phase of the FFT?
           
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Phase (radians)')
  return plt

In [ ]:
myPlotFFTPhase(h_iir, x_lim=40000, fs=fs11*7)

In [ ]:
myPlotFFTPhase(h_hann, n_fft=4096, x_lim=40000, fs=fs11*7)

# Is there an optimal or ideal FIR low-pass filter?



In [ ]:
fs = fs11*7
N = 1024
L = 32
H_ideal = np.zeros(N)
H_ideal[:L] = np.ones(L)

fig = plt.figure(figsize=(16,4))
f = np.arange(N) * fs / N
plt.plot(f, H_ideal)

## Is there an *inverse* Fourier Transform?
We haven't learned this yet, but let's try it...

In [ ]:
h_ideal = np.fft.ifft(H_ideal)
myPlot(np.real(h_ideal), fs=fs11*7)

In [ ]:
myPlot(np.imag(h_ideal), fs=fs11*7)

In [ ]:
H_ideal_sym = np.append(H_ideal, np.flipud(H_ideal))

fig = plt.figure(figsize=(16,4))
f = np.arange(len(H_ideal_sym)) * fs / len(H_ideal_sym)
plt.plot(f, H_ideal_sym)

In [ ]:
h_ideal_sym = np.fft.ifft(H_ideal_sym)
myPlot(np.real(h_ideal_sym))

In [ ]:
h_ideal_shift = np.fft.fftshift(h_ideal_sym)
myPlot(np.real(h_ideal_shift))